Importando as bibliotecas

In [1]:
import pandas as pd
import chardet
import unidecode

In [2]:
# 1. Identificar o encoding do arquivo

with open('natal2021.csv', 'rb') as f:
    result = chardet.detect(f.read())
encoding = result['encoding']


In [3]:
# 2. Processar o arquivo de 1000 em 1000 linhas

chunk_size = 1000
chunks = pd.read_csv('natal2021.csv', chunksize=chunk_size, encoding=encoding)


In [4]:
# 3. Remover os espaços das colunas

def remove_spaces(df):
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()
    return df


In [5]:
# 4. Criar uma coluna CITY_ASCII no arquivo

def create_ascii_col(df):
    df['CITY_ASCII'] = df['CITY'].apply(lambda x: unidecode.unidecode(x).upper())
    return df


In [6]:
# 5. Remover os caracteres não numéricos da coluna PHONE, os - e os ()

def remove_non_numeric(df):
    df['PHONE'] = df['PHONE'].str.replace('-', '').str.replace('(', '').str.replace(')', '').str.replace(r'\D+', '', regex=True)
    return df
  

In [ ]:
#6. Aplicar as funções a cada chunk e salvar em um novo arquivo CSV

with open('arquivo_tratado_natal2021.csv', 'w', encoding='utf-8', newline='') as f:
    for i, chunk in enumerate(chunks):
        print(f'Processing chunk {i+1}')
        chunk = remove_spaces(chunk)
        chunk = create_ascii_col(chunk)
        chunk = remove_non_numeric(chunk)
        chunk.to_csv(f, header=(i == 0), index=True)
print('Processing complete!')
